## Model 1
In this model in our two-model layer, we start with input the recent historical stats of the teams in a game and predict the stats for the team in their pending matchup. 

In [8]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import Ridge
pd.options.mode.chained_assignment = None


In [9]:
games_df = pd.read_csv("clean_games_with_opp.csv", index_col=0)
games_df.reset_index(drop=True, inplace=True)
games_df.head()

,school,school_id,points,fumblesRecovered,rushingTDs,passingTDs,kickReturnYards,kickReturnTDs,kickReturns,kickingPoints,...,opposing_rushingTDs,opposing_points,side,thirdDownConverts,thirdDownAttempts,fourthDownConverts,fourthDownAttempts,completions,passAttempts,passCompletionPercentage
0,Navy,2426,17,0.0,2.0,0.0,64.0,0.0,4.0,5.0,...,1.0,34,A,4.0,12.0,1.0,1.0,2.0,4.0,0.500000
1,UAB,5,48,1.0,4.0,2.0,64.0,0.0,2.0,12.0,...,1.0,10,A,9.0,14.0,0.0,0.0,13.0,20.0,0.650000
2,UMass,113,7,0.0,0.0,1.0,95.0,0.0,4.0,1.0,...,2.0,30,A,3.0,11.0,1.0,1.0,9.0,22.0,0.409091
3,UCF,2116,24,1.0,2.0,1.0,142.0,0.0,5.0,6.0,...,1.0,26,A,5.0,13.0,2.0,2.0,12.0,22.0,0.545455
4,Houston,248,7,1.0,1.0,0.0,89.0,0.0,6.0,1.0,...,3.0,27,A,4.0,16.0,4.0,4.0,25.0,50.0,0.500000


In [10]:
# Dropping all columns with more than 25 nulls, then dropping the individual rows that have nulls
x = games_df.isna().sum().to_dict()
column_drop = []

for k, v in x.items():
    if v > 25 :
        column_drop.append(k)

# Additionally, these columns are weird formats or already covered by other variables (ie the possession stuff)
extend_list = ['completionAttempts','totalPenaltiesYards','possessionTime','possession_minutes','possession_seconds','year','week','school','opposing_points']

#column_drop.extend(extend_list)
column_drop = column_drop + extend_list

games_df.drop(columns=column_drop, inplace = True)
games_df.dropna(axis = 0, inplace = True)

games_df = games_df.select_dtypes(exclude=["object"])

games_df = games_df.sort_values("id")

In [11]:
games_df.keys()

Index(['school_id', 'points', 'fumblesRecovered', 'rushingTDs', 'passingTDs',
       'firstDowns', 'thirdDownEff', 'totalYards', 'netPassingYards',
       'yardsPerPass', 'rushingYards', 'rushingAttempts',
       'yardsPerRushAttempt', 'turnovers', 'fumblesLost', 'interceptions',
       'poss_total_sec', 'id', 'opposing_netPassingYards',
       'opposing_yardsPerPass', 'opposing_rushingYards',
       'opposing_yardsPerRushAttempt', 'opposing_passingTDs',
       'opposing_rushingTDs', 'thirdDownConverts', 'thirdDownAttempts',
       'fourthDownConverts', 'fourthDownAttempts', 'completions',
       'passAttempts', 'passCompletionPercentage'],
      dtype='object')

We initially set the historical memory equal to 12 games. This is roughly the past seasons-worth of games for an average team. 

In [12]:
memory = 12

In [13]:
teams = set(games_df["school_id"])
print(teams)
print(len(teams))

{2, 2050, 5, 6, 2567, 8, 2569, 9, 2571, 2572, 12, 13, 16, 2065, 2579, 21, 2582, 23, 24, 25, 26, 3101, 30, 2083, 36, 2084, 38, 2598, 41, 43, 47, 48, 2097, 50, 52, 55, 57, 58, 2619, 59, 61, 2110, 2617, 2623, 62, 66, 2115, 2116, 68, 2628, 2117, 70, 2633, 2630, 2635, 2636, 77, 2638, 79, 2127, 2641, 2634, 2643, 2132, 84, 2640, 87, 2649, 2653, 93, 2655, 96, 97, 98, 99, 2142, 103, 107, 113, 2674, 2678, 119, 120, 2169, 2681, 127, 130, 2692, 2181, 135, 2184, 142, 145, 2193, 147, 2197, 150, 151, 152, 153, 154, 2711, 2199, 2717, 158, 2710, 160, 155, 2210, 164, 166, 167, 2729, 2226, 2229, 2230, 183, 2747, 189, 2751, 193, 194, 2755, 2241, 197, 2754, 2247, 195, 201, 202, 204, 2771, 213, 2261, 218, 221, 222, 227, 228, 2277, 231, 233, 235, 236, 238, 239, 149, 2287, 242, 2803, 245, 2294, 248, 249, 2296, 251, 252, 253, 254, 256, 2305, 258, 259, 257, 2309, 2306, 264, 265, 2320, 275, 276, 277, 278, 2837, 2329, 282, 2335, 290, 2341, 295, 2348, 301, 302, 304, 2198, 309, 311, 322, 324, 326, 328, 2377, 331, 3

In [7]:
X = []
y = []
for team in teams:
    team_df = games_df[games_df["school_id"] == team]
    team_df = team_df.sort_values("id")
    team_df.drop(columns=["id", "school_id"], inplace=True)
    team_array = np.array(team_df)
    
    for i in range(memory, len(team_array)):
        X.append(np.hstack(team_array[i-memory:i]))
        y.append(team_array[i])
   

We now split the data into the training and testing sets:

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Now we have the train and test data, we will scale the input to the model. Since the output is stats each working on the same scales as the inputs, we scale the output as well.

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

scaler2 = StandardScaler()
scaler2.fit(y_train)
y_train_scaled = scaler2.transform(y_train)
y_test_scaled = scaler2.transform(y_test)

## Training and testing models
We first try some linear models for predicting the stats in the next game.

In [11]:
from sklearn.linear_model import Ridge, ElasticNet, Lasso, LinearRegression

reg25 = Ridge(alpha=0.25)
reg25.fit(X_train_scaled, y_train_scaled)

reg50 = Ridge(alpha=0.5)
reg50.fit(X_train_scaled, y_train_scaled)

reg100 = Ridge(alpha=1)
reg100.fit(X_train_scaled, y_train_scaled)

Ridge(alpha=1)

We now make predictions with the ridge models on the test set and compute the mean squared error. 

In [12]:
from sklearn.metrics import mean_squared_error

pred_scores25 = reg25.predict(X_test_scaled)
pred_scores50 = reg50.predict(X_test_scaled)
pred_scores100 = reg100.predict(X_test_scaled)

print(f"alpha: MSE are: \n 0.25:{mean_squared_error(pred_scores25, y_test_scaled)} \n 0.50:{mean_squared_error(pred_scores50, y_test_scaled)} \n 1.00:{mean_squared_error(pred_scores100, y_test_scaled)}")

alpha: MSE are: 
 0.25:0.9313190372921086 
 0.50:0.9313196486549717 
 1.00:0.9312959331776132


In [13]:
lasso = Lasso(alpha=1)
lasso.fit(X_train_scaled, y_train_scaled)

ela_net = ElasticNet(alpha=1)
ela_net.fit(X_train_scaled, y_train_scaled)

print(f"MSE are \n Lasso:{mean_squared_error(lasso.predict(X_test_scaled), y_test_scaled)}\n ElasticNet:{mean_squared_error(ela_net.predict(X_test_scaled), y_test_scaled)}")

MSE are 
 Lasso:0.9949641965238859
 ElasticNet:0.9949641965238859


In the previous modelling attempts, we predicted the stats for a given team based only on that team's history. Now we will try to look at predicting stats for a given game based on the combined histories of the two teams playing.

In [16]:
teams_dict = {}
for team in teams:
    temp_df = games_df[games_df["school_id"] == team].sort_values("id")
    #temp_df.reset_index(inplace=True)
    teams_dict[team] = temp_df.reset_index(drop=True)


In [15]:
X2 = []
y2 = []
for team in teams:
    team1_df = teams_dict[team]
    print(f"on team {team}")
    
    # loop over all possible games for a given team with enough prior games
    for i in range(memory, len(team1_df)):
        # find the two teams participating in the game
        game_id = team1_df.loc[i]["id"]
        both_teams_ids = games_df[games_df["id"] == game_id]["school_id"]
        both_teams_ids = list(both_teams_ids)

        # throw out some odd cases where  there was only data for one of the teams in a given game
        if len(both_teams_ids) != 2:
            continue
        #print(f"both teams are {both_teams_ids}")
 
        # extract the dataframes for the two teams, find which game it was for them
        t1_df = teams_dict[both_teams_ids[0]]#.loc[i-memory:i+1]
        t2_df = teams_dict[both_teams_ids[1]]#.loc[i-memory:i+1]

        t1_gamen = np.where(t1_df["id"] == game_id)[0][0]
        t2_gamen = np.where(t2_df["id"] == game_id)[0][0]

        # check to make sure that both teams have enough games in the past
        if t1_gamen < memory or t2_gamen < memory:
            continue
        
        # extract the games in recent history, dropping columns that don't go into the data
        t1_df = t1_df.loc[t1_gamen-memory:t1_gamen]#+1]
        t2_df = t2_df.loc[t2_gamen-memory:t2_gamen]#+1]
        t1_df.drop(columns=["id", "school_id"], inplace=True)
        t2_df.drop(columns=["id", "school_id"], inplace=True)

        t1_array = np.array(t1_df)
        t2_array = np.array(t2_df)

        # break the arrays into past games and current game to go into training and test set
        t1_past = t1_array[0:-1]
        t1_present = t1_array[-1]
        t2_past = t2_array[0:-1]
        t2_present = t2_array[-1]


        X2.append(np.hstack([t1_past, t2_past]))
        y2.append(np.hstack([t1_present, t2_present]))

on team 2
on team 2050
on team 5
on team 6
on team 2567
on team 8
on team 2569
on team 9
on team 2571
on team 2572
on team 12
on team 13
on team 16
on team 2065
on team 2579
on team 21
on team 2582
on team 23
on team 24
on team 25
on team 26
on team 3101
on team 30
on team 2083
on team 36
on team 2084
on team 38
on team 2598
on team 41
on team 43
on team 47
on team 48
on team 2097
on team 50
on team 52
on team 55
on team 57
on team 58
on team 2619
on team 59
on team 61
on team 2110
on team 2617
on team 2623
on team 62
on team 66
on team 2115
on team 2116
on team 68
on team 2628
on team 2117
on team 70
on team 2633
on team 2630
on team 2635
on team 2636
on team 77
on team 2638
on team 79
on team 2127
on team 2641
on team 2634
on team 2643
on team 2132
on team 84
on team 2640
on team 87
on team 2649
on team 2653
on team 93
on team 2655
on team 96
on team 97
on team 98
on team 99
on team 2142
on team 103
on team 107
on team 113
on team 2674
on team 2678
on team 119
on team 120
on team 216

In [16]:
X2_flat = []
for i in range(len(X2)):
    X2_flat.append(X2[i].reshape(-1))

In [17]:
X2_flat = np.array(X2_flat)
y2 = np.array(y2)
print(X2_flat.shape, y2.shape)

(12742, 672) (12742, 56)


Now we scale the data, split it into train and test sets, and then train and test various models.

In [18]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_flat, y2, test_size=0.25, random_state=42)


scaler = StandardScaler()
scaler.fit(X2_train)
X2_train_scaled = scaler.transform(X2_train)
X2_test_scaled = scaler.transform(X2_test)

scaler2 = StandardScaler()
scaler2.fit(y2_train)
y2_train_scaled = scaler2.transform(y2_train)
y2_test_scaled = scaler2.transform(y2_test)

In [19]:
reg25 = Ridge(alpha=0.25)
reg25.fit(X2_train_scaled, y2_train_scaled)

reg50 = Ridge(alpha=0.5)
reg50.fit(X2_train_scaled, y2_train_scaled)

reg100 = Ridge(alpha=1)
reg100.fit(X2_train_scaled, y2_train_scaled)

Ridge(alpha=1)

In [20]:
pred_scores25 = reg25.predict(X2_test_scaled)
pred_scores50 = reg50.predict(X2_test_scaled)
pred_scores100 = reg100.predict(X2_test_scaled)

print(f"alpha: MSE are: \n 0.25:{mean_squared_error(pred_scores25, y2_test_scaled)} \n 0.50:{mean_squared_error(pred_scores50, y2_test_scaled)} \n 1.00:{mean_squared_error(pred_scores100, y2_test_scaled)}")

alpha: MSE are: 
 0.25:0.856699449691887 
 0.50:0.8567345821663706 
 1.00:0.8566922650408936


We see that the performance is improved by considering the historical data of both teams, not just each team independently.

## Neural Network Approach with LSTM
We now develop a model utilizing neural networks. The model will take in the recent historical data of the two teams and output the predicted stats of the next game. We utilize an LSTM model in our approach, a standard technique when dealing with time-varying inputs as we have.

In [17]:
Xn = []
yn = []
for team in teams:
    team1_df = teams_dict[team]
    print(f"on team {team}")
    
    # loop over all possible games for a given team with enough prior games
    for i in range(memory, len(team1_df)):
        # find the two teams participating in the game
        game_id = team1_df.loc[i]["id"]
        both_teams_ids = games_df[games_df["id"] == game_id]["school_id"]
        both_teams_ids = list(both_teams_ids)

        # throw out some odd cases where  there was only data for one of the teams in a given game
        if len(both_teams_ids) != 2:
            continue
        #print(f"both teams are {both_teams_ids}")
 
        # extract the dataframes for the two teams, find which game it was for them
        t1_df = teams_dict[both_teams_ids[0]]#.loc[i-memory:i+1]
        t2_df = teams_dict[both_teams_ids[1]]#.loc[i-memory:i+1]

        t1_gamen = np.where(t1_df["id"] == game_id)[0][0]
        t2_gamen = np.where(t2_df["id"] == game_id)[0][0]

        # check to make sure that both teams have enough games in the past
        if t1_gamen < memory or t2_gamen < memory:
            continue
        
        # extract the games in recent history, dropping columns that don't go into the data
        t1_df = t1_df.loc[t1_gamen-memory:t1_gamen]#+1]
        t1_points = t1_df["points"].to_numpy()[-1]
        t2_df = t2_df.loc[t2_gamen-memory:t2_gamen]#+1]
        t2_points = t2_df["points"].to_numpy()[-1]
        t1_df.drop(columns=["id", "school_id", "points"], inplace=True)
        t2_df.drop(columns=["id", "school_id", "points"], inplace=True)

        t1_array = np.array(t1_df)
        t2_array = np.array(t2_df)

        # break the arrays into past games and current game to go into training and test set
        t1_past = t1_array[0:-1]
        #t1_present = t1_array[-1]
        t2_past = t2_array[0:-1]
        #t2_present = t2_array[-1]


        Xn.append(np.hstack([t1_past, t2_past]))
        yn.append(np.array([t1_points, t2_points]))

on team 2
on team 2050
on team 5
on team 6
on team 2567
on team 8
on team 2569
on team 9
on team 2571
on team 2572
on team 12
on team 13
on team 16
on team 2065
on team 2579
on team 21
on team 2582
on team 23
on team 24
on team 25
on team 26
on team 3101
on team 30
on team 2083
on team 36
on team 2084
on team 38
on team 2598
on team 41
on team 43
on team 47
on team 48
on team 2097
on team 50
on team 52
on team 55
on team 57
on team 58
on team 2619
on team 59
on team 61
on team 2110
on team 2617
on team 2623
on team 62
on team 66
on team 2115
on team 2116
on team 68
on team 2628
on team 2117
on team 70
on team 2633
on team 2630
on team 2635
on team 2636
on team 77
on team 2638
on team 79
on team 2127
on team 2641
on team 2634
on team 2643
on team 2132
on team 84
on team 2640
on team 87
on team 2649
on team 2653
on team 93
on team 2655
on team 96
on team 97
on team 98
on team 99
on team 2142
on team 103
on team 107
on team 113
on team 2674
on team 2678
on team 119
on team 120
on team 216

In [18]:
Xn = np.array(Xn)
yn = np.array(yn)
print(Xn.shape, yn.shape)

(12742, 12, 56) (12742, 2)


In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Xn_train, Xn_test, yn_train, yn_test = train_test_split(Xn, yn, test_size=0.25, random_state=42)

n_train_samples = len(Xn_train)
n_test_samples = len(Xn_test)

scaler = StandardScaler()
Xn_train_flat = np.reshape(Xn_train, (n_train_samples, -1))
Xn_test_flat = np.reshape(Xn_test, (n_test_samples, -1))
scaler.fit(Xn_train_flat)
Xn_train_scaled = scaler.transform(Xn_train_flat)
Xn_test_scaled = scaler.transform(Xn_test_flat)
Xn_train_scaled = np.reshape(Xn_train_scaled, (n_train_samples, memory, -1))
Xn_test_scaled = np.reshape(Xn_test_scaled, (n_test_samples, memory, -1))

scaler2 = StandardScaler()
scaler2.fit(yn_train)
yn_train_scaled = scaler2.transform(yn_train)
yn_test_scaled = scaler2.transform(yn_test)

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

model = Sequential()

model.add(LSTM(units=Xn.shape[2], return_sequences=True, input_shape=(Xn_train_scaled.shape[1], Xn_train_scaled.shape[2])))
model.add(LSTM(units=Xn.shape[1]))
model.add(Dense(units=yn.shape[1]))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 12, 56)            25312     
                                                                 
 lstm_4 (LSTM)               (None, 12)                3312      
                                                                 
 dense_2 (Dense)             (None, 2)                 26        
                                                                 
Total params: 28650 (111.91 KB)
Trainable params: 28650 (111.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.compile(loss="mean_squared_error", optimizer="adam")

In [26]:
model.fit(Xn_train_scaled, yn_train_scaled, epochs=100, verbose=1)

Epoch 1/100

299/299 [==============================] - 6s 8ms/step - loss: 0.7708
Epoch 2/100
299/299 [==============================] - 2s 7ms/step - loss: 0.7162
Epoch 3/100
299/299 [==============================] - 2s 7ms/step - loss: 0.6698
Epoch 4/100
299/299 [==============================] - 2s 8ms/step - loss: 0.6158
Epoch 5/100
299/299 [==============================] - 2s 8ms/step - loss: 0.5486
Epoch 6/100
299/299 [==============================] - 2s 8ms/step - loss: 0.4804
Epoch 7/100
299/299 [==============================] - 2s 8ms/step - loss: 0.4186
Epoch 8/100
299/299 [==============================] - 2s 8ms/step - loss: 0.3612
Epoch 9/100
299/299 [==============================] - 2s 8ms/step - loss: 0.3164
Epoch 10/100
299/299 [==============================] - 2s 8ms/step - loss: 0.2757
Epoch 11/100
299/299 [==============================] - 2s 8ms/step - loss: 0.2401
Epoch 12/100
299/299 [==============================] - 2s 8ms/step - loss: 0.2130
Epoch 13/100

In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

model3 = Sequential()

model3.add(LSTM(units=Xn.shape[2], return_sequences=True, input_shape=(Xn_train_scaled.shape[1], Xn_train_scaled.shape[2])))
model3.add(Dropout(0.2))

model3.add(LSTM(units=Xn.shape[2], return_sequences=True))
model3.add(Dropout(0.2))

model3.add(LSTM(units=Xn.shape[2]))
model3.add(Dropout(0.2))

model3.add(Dense(units=yn.shape[1]))

model3.summary()
model3.compile(loss="mean_squared_error", optimizer="adam")

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 12, 56)            25312     
                                                                 
 dropout_6 (Dropout)         (None, 12, 56)            0         
                                                                 
 lstm_13 (LSTM)              (None, 12, 56)            25312     
                                                                 
 dropout_7 (Dropout)         (None, 12, 56)            0         
                                                                 
 lstm_14 (LSTM)              (None, 56)                25312     
                                                                 
 dropout_8 (Dropout)         (None, 56)                0         
                                                                 
 dense_5 (Dense)             (None, 2)                

In [47]:
model3.fit(Xn_train_scaled, yn_train_scaled, epochs=200, verbose=True)

Epoch 1/200
299/299 [==============================] - 9s 12ms/step - loss: 0.7969
Epoch 2/200
299/299 [==============================] - 4s 13ms/step - loss: 0.7378
Epoch 3/200
299/299 [==============================] - 4s 13ms/step - loss: 0.7009
Epoch 4/200
299/299 [==============================] - 4s 13ms/step - loss: 0.6445
Epoch 5/200
299/299 [==============================] - 4s 13ms/step - loss: 0.5838
Epoch 6/200
299/299 [==============================] - 4s 13ms/step - loss: 0.5176
Epoch 7/200
299/299 [==============================] - 4s 13ms/step - loss: 0.4519
Epoch 8/200
299/299 [==============================] - 4s 13ms/step - loss: 0.4031
Epoch 9/200
299/299 [==============================] - 4s 13ms/step - loss: 0.3538
Epoch 10/200
299/299 [==============================] - 4s 13ms/step - loss: 0.3139
Epoch 11/200
299/299 [==============================] - 4s 13ms/step - loss: 0.2791
Epoch 12/200
299/299 [==============================] - 4s 13ms/step - loss: 0.2493
E

In [52]:
yn_pred = model3.predict(Xn_test_scaled)
print(mean_squared_error(yn_test_scaled, yn_pred))

100/100 [==============================] - 1s 7ms/step
0.31027311508729793


In [51]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

yn_pred = model3.predict(Xn_test_scaled)
yn_pred = scaler2.inverse_transform(yn_pred)
print(mean_squared_error(yn_test, yn_pred))
print(mean_absolute_error(yn_test, yn_pred))

100/100 [==============================] - 1s 5ms/step
62.04484006732192
3.9937795180449123


In [49]:
print(yn_pred)
print(yn_test)

[[10.031242  9.477807]
 [33.32205  46.80371 ]
 [36.354    24.018084]
 ...
 [34.021908 12.794992]
 [ 9.149614 27.79305 ]
 [12.484794  4.638334]]
[[13  9]
 [31 48]
 [37 24]
 ...
 [35 13]
 [20 37]
 [14  6]]


In [50]:
real_games = []
pred_games = []
for i in range(len(yn_test)):
    if yn_test[i][0] > yn_test[i][1]:
        real_games.append(1)
    else:
        real_games.append(0)
    if yn_pred[i][0] > yn_pred[i][1]:
        pred_games.append(1)
    else:
        pred_games.append(0)

incorrect_games = np.sum(np.abs(np.array(pred_games) - np.array(real_games)))
correct_games = len(yn_test) - incorrect_games
print(f"correct games is: {correct_games} out of {len(yn_test)}, pct correct is {(correct_games)/(correct_games + incorrect_games)}.")

correct games is: 2886 out of 3186, pct correct is 0.9058380414312618.


In [27]:
Xn = np.array(Xn)
yn = np.array(yn)
print(Xn.shape, yn.shape)

(12742, 12, 56) (12742, 56)


In [28]:
Xn_train, Xn_test, yn_train, yn_test = train_test_split(Xn, yn, test_size=0.25, random_state=42)

n_train_samples = len(Xn_train)
n_test_samples = len(Xn_test)

scaler = StandardScaler()
Xn_train_flat = np.reshape(Xn_train, (n_train_samples, -1))
Xn_test_flat = np.reshape(Xn_test, (n_test_samples, -1))
scaler.fit(Xn_train_flat)
Xn_train_scaled = scaler.transform(Xn_train_flat)
Xn_test_scaled = scaler.transform(Xn_test_flat)
Xn_train_scaled = np.reshape(Xn_train_scaled, (n_train_samples, memory, -1))
Xn_test_scaled = np.reshape(Xn_test_scaled, (n_test_samples, memory, -1))

scaler2 = StandardScaler()
scaler2.fit(yn_train)
yn_train_scaled = scaler2.transform(yn_train)
yn_test_scaled = scaler2.transform(yn_test)

In [34]:
model = Sequential()

model.add(LSTM(units=yn.shape[1], input_shape=(Xn_train_scaled.shape[1], Xn_train_scaled.shape[2])))
model.add(Dense(units=yn.shape[1]))

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 56)                25312     
                                                                 
 dense (Dense)               (None, 56)                3192      
                                                                 
Total params: 28504 (111.34 KB)
Trainable params: 28504 (111.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
model.compile(loss="mean_squared_error", optimizer="adam")

In [37]:
model.fit(Xn_train_scaled, yn_train_scaled, epochs=100, verbose=1)

Epoch 1/100

299/299 [==============================] - 4s 5ms/step - loss: 0.8993
Epoch 2/100
299/299 [==============================] - 2s 5ms/step - loss: 0.8450
Epoch 3/100
299/299 [==============================] - 1s 5ms/step - loss: 0.8290
Epoch 4/100
299/299 [==============================] - 2s 5ms/step - loss: 0.8150
Epoch 5/100
299/299 [==============================] - 1s 5ms/step - loss: 0.8006
Epoch 6/100
299/299 [==============================] - 2s 5ms/step - loss: 0.7856
Epoch 7/100
299/299 [==============================] - 1s 5ms/step - loss: 0.7691
Epoch 8/100
299/299 [==============================] - 2s 5ms/step - loss: 0.7520
Epoch 9/100
299/299 [==============================] - 1s 5ms/step - loss: 0.7340
Epoch 10/100
299/299 [==============================] - 2s 5ms/step - loss: 0.7166
Epoch 11/100
299/299 [==============================] - 1s 5ms/step - loss: 0.6995
Epoch 12/100
299/299 [==============================] - 2s 5ms/step - loss: 0.6839
Epoch 13/100

In [38]:
yn_pred = model.predict(Xn_test_scaled)

100/100 [==============================] - 1s 3ms/step


In [39]:
print(mean_squared_error(yn_test_scaled, yn_pred))

0.7059745207355442


In [46]:
diffs = scaler2.inverse_transform(yn_test_scaled) - scaler2.inverse_transform(yn_pred)

In [48]:
diffs_df = pd.DataFrame(diffs[:,0:28], columns=t1_df.keys())

In [44]:
len(t1_df.keys())

28

In [50]:
diffs_df

,fumblesRecovered,rushingTDs,passingTDs,firstDowns,thirdDownEff,totalYards,netPassingYards,yardsPerPass,rushingYards,rushingAttempts,...,opposing_yardsPerRushAttempt,opposing_passingTDs,opposing_rushingTDs,thirdDownConverts,thirdDownAttempts,fourthDownConverts,fourthDownAttempts,completions,passAttempts,passCompletionPercentage
0,-0.787471,-0.623071,-1.280493,-2.716593,0.160488,-61.874268,-20.760956,-0.059482,-41.374687,2.470778,...,-0.832166,-1.716744,-2.261754,2.454082,0.619487,-1.669718,-1.669718,-2.460936,-2.634693,-0.038308
1,0.135716,2.649791,-1.220105,2.644871,0.058802,17.996979,12.228912,0.728169,5.655609,0.155582,...,-1.429580,-2.030435,0.754238,0.425841,-0.895601,-1.386396,-1.386395,2.097589,-4.476471,0.141760
2,-1.221952,0.919461,-0.049857,1.063900,0.137590,4.400085,30.666962,3.139234,-26.323242,-6.516758,...,-0.916096,0.437135,-0.168412,-2.265317,-7.199229,0.007008,0.007009,-0.148223,-5.221842,0.121046
3,0.101742,2.602601,0.360394,2.525581,0.006127,66.552429,42.580994,-0.299557,24.054413,-2.735428,...,-0.167954,-0.303723,1.000788,-0.222930,-0.223150,-0.205026,-0.205026,4.234775,7.268991,0.045232
4,-1.509203,0.247883,-0.566114,-0.691948,0.003632,-1.638763,21.916595,1.549952,-23.377289,-8.506020,...,-0.246606,-0.216064,-2.185653,-0.780818,-2.056944,-2.504359,-2.504359,-2.251232,-2.164650,-0.024807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3181,-0.732223,0.221433,-0.821900,-1.479908,-0.097866,-6.562836,-6.710770,-0.000724,0.279190,3.676392,...,-1.217415,-0.866635,-0.786627,-1.600833,-1.414748,-0.764393,-0.764393,1.462877,-0.792961,0.063101
3182,0.442357,1.207644,-0.370953,1.774637,0.145565,-14.824585,-1.131027,-1.302375,-13.914894,6.366629,...,0.355629,-1.019268,0.077661,4.356813,5.663538,1.373217,1.373217,3.492224,4.312843,0.031985
3183,0.074386,-1.306584,0.075787,-2.692142,-0.101866,-47.222565,54.809769,0.222544,-101.785736,-6.917194,...,-0.965228,-0.251059,0.099351,-0.951893,0.598404,-0.669117,-0.669117,0.759758,4.773159,-0.079777
3184,0.316352,-0.991813,1.125704,0.162011,0.094831,66.242218,118.429260,2.101486,-52.163979,-11.531380,...,-0.299980,-0.396057,0.554286,1.803398,1.623865,1.068068,1.068068,4.942402,9.621449,0.015711
